In [ ]:
import collections
import pandas as pd
import ast
import json
import numpy as np
import re
pd.set_option('display.max_columns', np.nan)

In [ ]:
#read dataset
df = pd.read_csv('igb_databaselastnoslug.csv')
#creating lists 
df_names = []

In [ ]:
#clean columns for merging
#remove spaces and keep only alphanumeric
for index,i in df.iterrows():
    try:
        df_names.append((re.sub(r'\W+', '',i['name'])))
    except:
        df_names.append(i['name'])
#insert it as columns
df = df.assign(new_name = df_names)

In [ ]:
#drop duplicated on new_name column
df.drop_duplicates(subset=['new_name'],inplace=True)
df.reset_index(inplace=True)
df.columns

In [ ]:
#replacing '' as missing
df.replace('',np.nan)

In [ ]:
#reset index
df.reset_index(inplace=True)
#checking missing values
df.isna().sum()

In [ ]:
#functions to clean some columns, based on the methodology of needed for each column

def filiterlists(columns_df):
    returned_list = []
    for i in columns_df:
        words = ''
        if(i == '' or pd.isnull(i)):
            break
        cool = ast.literal_eval(i)
        for i in cool:
            s = str(i).replace('\'','\"').replace('[','').replace(']','')
            word = json.loads(s)['name']
            words += word + ','
        returned_list.append(words[:-1])
    return returned_list
#method for simillar columns that needs cleaning
def filter_simillar(column,dataframe):
    list = []
    for i in dataframe[column]:
        try:
            row = ast.literal_eval(i)
            words = ''
            for ob in row:
                words += ob['name'] + ','
            list.append(words[:-1])
        except:
            list.append(np.nan)
    return list

In [ ]:
#%%
#cleaning columns from IGDB database
#clean game_modes column
game_modes = filiterlists(df['game_modes'])
#clean involed_companies column
involed_companies = []
for i in df['involved_companies']:
    try:
        row = ast.literal_eval(i)
        words = ''
        for ob in row:
            name = ob['company']
            words += name['name'] + ','
        involed_companies.append(words[:-1])
    except:
        involed_companies.append(np.nan)

#clean age_rating column
age_rating = []
for index,i in df.iterrows():
    try:
        words = ''
        for i in ast.literal_eval(i['age_ratings']):
            for i in i['content_descriptions']:
                words += i['description'] + ','
        age_rating.append(words[:-1])
    except:
        age_rating.append(np.nan)
#clean player_perspectives column
player_perspectives = []
for index,i in df.iterrows():
    try:
        words = ''
        for i in ast.literal_eval(i['player_perspectives']):
            words += i['name'] + ','
        player_perspectives.append(words[:-1])
    except:
        player_perspectives.append(np.nan)
#clean franchises column
franchises = []
for index,i in df.iterrows():
    try:
        words = ''
        for i in ast.literal_eval(i['franchises']):
            for i in i['games']:
                words += i['name'] + ','
        franchises.append(words[:-1])
    except:
        franchises.append(np.nan)
#clean collection column
collection = []
for index,i in df.iterrows():
    try:
        words = ''
        for i in ast.literal_eval(i['collection'])['games']:
            words += i['name'] + ','
        collection.append(words[:-1])
    except:
        collection.append(np.nan)  
#clean external_games column
external_games = filter_simillar('external_games',df)
#clean genres column
genres = filter_simillar('genres',df)        
#clean platforms column
platforms = filter_simillar('platforms',df)
#clean similar_games column
similar_games = filter_simillar('similar_games',df)
#clean game_modes
game_modes = filter_simillar('game_modes',df)
#clean themes column
themes = filter_simillar('themes',df)
#clean game_engines column
game_engines = filter_simillar('game_engines',df)

In [ ]:
#drop old uncleaned columns
df.drop(columns=['platforms','similar_games','external_games','new_name','genres','index','themes','player_perspectives','similar_games',
'game_modes','age_ratings','involved_companies','game_engines','franchises','collection'],inplace=True)

In [ ]:
#insert new cleaned columns
df = df.assign(external_games = external_games,genres = genres,platforms = platforms,
similar_games = similar_games,themes=themes,player_perspectives=player_perspectives
,game_modes = game_modes,age_ratings=age_rating,involved_companies = involed_companies,
franchises = franchises, game_engines = game_engines,collection = collection)

In [ ]:
#drop redundent columns
df.drop(columns=['slug',],inplace=True)

In [ ]:
#id from int to string
df['id'] = df['id'].astype(str)

In [ ]:
#reorder columns only significant 
df = df[['name','id','summary','game_engines', 
         'collection',
       'storyline', 'franchises',
        'expansions', 'dlcs', 'franchise',
       'external_games', 'genres', 'platforms', 'similar_games', 'themes',
       'player_perspectives', 'game_modes',
       'involved_companies']]

In [ ]:
#check for missing values
df.isna().sum()

In [ ]:
# %%
df.head()

In [ ]:
#save dataframe as csv file
df.to_csv('IGDB.csv',encoding='utf-8',index=False)